# Capitulo 11: System Prompts Avanzados

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu API key y establecer la funcion auxiliar.

**Nota:** Los prompts de ejemplo se mantienen en ingles ya que las tecnicas de ingenieria de prompts son independientes del idioma y Claude responde de manera mas predecible en ingles.

In [ ]:
!pip install anthropic

import re
import json
import anthropic
import sys, os

# Asegurar compatibilidad de path
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar variables del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    messages = [{"role": "user", "content": prompt}]
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## Leccion

En el **Capitulo 1**, introdujimos los system prompts como una forma de dar contexto e instrucciones a Claude. En este capitulo, exploraremos tecnicas avanzadas para construir system prompts poderosos y bien estructurados.

### 1. System Prompts Multi-Seccion con Estructura XML

Los system prompts simples funcionan bien para tareas basicas, pero para comportamientos complejos, es mejor organizar el system prompt en **secciones claramente definidas** usando tags XML.

Esto ayuda a Claude a entender las diferentes dimensiones de su comportamiento esperado:

In [ ]:
# Ejemplo: System prompt multi-seccion
SYSTEM_PROMPT = """<role>
You are a senior financial analyst at a Fortune 500 company.
You have 20 years of experience in corporate finance and M&A.
</role>

<instructions>
- Always provide data-driven analysis
- Cite specific metrics when available
- Consider both risks and opportunities
- Use professional but accessible language
</instructions>

<output_format>
Structure every response as:
1. Executive Summary (2-3 sentences)
2. Detailed Analysis
3. Key Risks
4. Recommendations
</output_format>

<constraints>
- Never provide specific investment advice
- Always note that analysis is for informational purposes only
- If data is insufficient, clearly state what additional information is needed
</constraints>"""

PROMPT = "Analyze the potential impact of a 25% tariff on imported semiconductors for a US-based tech manufacturer."

print(get_completion(PROMPT, SYSTEM_PROMPT))

Observa como la respuesta de Claude sigue la estructura definida en `<output_format>` y respeta las restricciones de `<constraints>`. Las secciones XML actuan como **compartimentos mentales** que Claude puede consultar durante la generacion de su respuesta.

### 2. Definicion Detallada de Persona

Mas alla de un simple "You are a...", puedes definir personas con **multiples dimensiones**: experiencia, estilo de comunicacion, valores, y hasta limitaciones:

In [ ]:
# Persona detallada
SYSTEM_PROMPT = """<persona>
<name>Dr. Sarah Chen</name>
<background>PhD in Computer Science from MIT. 15 years in AI safety research. Former lead at DeepMind's alignment team.</background>
<communication_style>
- Uses analogies from everyday life to explain complex concepts
- Asks Socratic questions to guide understanding
- Acknowledges uncertainty openly
- Avoids jargon unless the user demonstrates familiarity
</communication_style>
<values>
- Intellectual honesty over reassurance
- Nuanced thinking over simplistic answers
- Teaching over lecturing
</values>
</persona>"""

PROMPT = "Can you explain what machine learning alignment is and why it matters?"

print(get_completion(PROMPT, SYSTEM_PROMPT))

### 3. Guardrails de Comportamiento

Los guardrails son **reglas que definen lo que Claude debe y no debe hacer**. Son especialmente importantes para aplicaciones de produccion:

In [ ]:
# System prompt con guardrails
SYSTEM_PROMPT = """<role>You are a children's educational assistant for ages 6-12.</role>

<allowed_topics>
- Science, math, history, geography, language arts
- Age-appropriate current events
- Creative writing and storytelling
- Study skills and learning strategies
</allowed_topics>

<forbidden_topics>
- Violence, weapons, or warfare details
- Adult content of any kind
- Political opinions or partisan content
- Medical advice beyond "ask your parents or doctor"
- Personal information requests
</forbidden_topics>

<behavioral_rules>
- If asked about a forbidden topic, say: "That's a great question to discuss with your parents or teacher! Let's talk about [redirect to related allowed topic]."
- Always encourage curiosity and asking questions
- Use simple vocabulary appropriate for the age group
- Include fun facts when relevant
</behavioral_rules>"""

# Probemos con un tema apropiado
print("=== Tema apropiado ===")
print(get_completion("Why do volcanoes erupt?", SYSTEM_PROMPT))

print("\n" + "="*60 + "\n")

# Probemos con un tema fuera de los limites
print("=== Tema fuera de limites ===")
print(get_completion("How do nuclear weapons work?", SYSTEM_PROMPT))

### 4. Formato de Output via System Prompts

Puedes usar system prompts para forzar formatos de output especificos, como JSON:

In [ ]:
# Forzar output JSON
SYSTEM_PROMPT = """You are a data extraction API. You MUST respond ONLY with valid JSON, no other text.

For any user input, extract the following fields into a JSON object:
{
  "entities": [{"name": string, "type": "person|organization|location|date|other"}],
  "sentiment": "positive|negative|neutral|mixed",
  "topics": [string],
  "language": string,
  "word_count": number
}

Rules:
- Always return valid JSON
- If a field cannot be determined, use null
- Never include explanations or text outside the JSON"""

PROMPT = "Apple Inc. announced record earnings yesterday in Cupertino. CEO Tim Cook expressed optimism about the company's future in AI."

response = get_completion(PROMPT, SYSTEM_PROMPT, prefill="{")
response = "{" + response  # Agregar el prefill de vuelta
print(response)

# Verificar que sea JSON valido
try:
    parsed = json.loads(response)
    print("\nJSON valido!", json.dumps(parsed, indent=2))
except json.JSONDecodeError as e:
    print(f"\nError de JSON: {e}")

### 5. System Prompt vs User Prompt: Cuando usar cada uno

| Aspecto | System Prompt | User Prompt |
|---------|--------------|-------------|
| Proposito | Comportamiento persistente | Tarea especifica |
| Alcance | Toda la conversacion | Un turno |
| Ejemplo | "Eres un experto en X" | "Analiza este texto" |
| Prioridad | Establece el contexto base | Define la accion |

**Regla general**: Pon en el system prompt todo lo que quieres que se aplique **siempre** (rol, formato, restricciones). Pon en el user prompt lo que es **especifico a cada interaccion** (datos, preguntas, tareas concretas).

### 6. Consideraciones sobre la longitud del System Prompt

- System prompts **cortos** (<500 tokens): Bien para tareas simples y directas
- System prompts **medianos** (500-2000 tokens): Ideal para la mayoria de aplicaciones
- System prompts **largos** (>2000 tokens): Puede diluir la atencion de Claude

**Consejo**: Pon las instrucciones mas importantes al **principio** y al **final** del system prompt (efecto de primacia y recencia).

---

## Ejercicios
- [Ejercicio 11.1 - Bot de Atencion al Cliente](#ejercicio-111---bot-de-atencion-al-cliente)
- [Ejercicio 11.2 - Guardrails Educativos](#ejercicio-112---guardrails-educativos)
- [Ejercicio 11.3 - Formato JSON Forzado](#ejercicio-113---formato-json-forzado)

### Ejercicio 11.1 - Bot de Atencion al Cliente

Escribe un system prompt multi-seccion (usando tags XML) para un bot de atencion al cliente que:
1. Categorice las quejas automaticamente
2. Responda con empatia
3. Proporcione un numero de ticket
4. Estructure su respuesta con secciones claras

In [ ]:
# Escribe tu system prompt aqui
SYSTEM_PROMPT = "[Reemplaza este texto con tu system prompt multi-seccion]"

# Queja de prueba
PROMPT = "I ordered a laptop 3 weeks ago and it still hasn't arrived. I've tried calling support twice but nobody answers. This is unacceptable!"

response = get_completion(PROMPT, SYSTEM_PROMPT)

# Funcion de calificacion
def grade_exercise(text):
    has_category = bool(re.search(r'<categor|categor|shipping|delivery|delayed', text, re.IGNORECASE))
    has_structure = bool(re.search(r'ticket|reference|caso|numero', text, re.IGNORECASE))
    return has_category and has_structure

print(response)
print("\n--------------------------- CALIFICACION ---------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response))

Si quieres una pista, ejecuta la celda siguiente!

In [ ]:
from hints import exercise_11_1_hint; print(exercise_11_1_hint)

### Ejercicio 11.2 - Guardrails Educativos

Crea un system prompt para un asistente de matematicas para ninos que:
1. Solo responda preguntas de matematicas
2. Rechace educadamente temas no relacionados
3. Use un tono amigable y motivador
4. Nunca de la respuesta directa, solo guie al estudiante

In [ ]:
# Escribe tu system prompt aqui
SYSTEM_PROMPT = "[Reemplaza este texto]"

# Prueba con pregunta de matematicas
response_math = get_completion("What is 15% of 80?", SYSTEM_PROMPT)
print("=== Pregunta de matematicas ===")
print(response_math)

# Prueba con tema no relacionado
response_off = get_completion("Tell me about dinosaurs", SYSTEM_PROMPT)
print("\n=== Tema no relacionado ===")
print(response_off)

# Funcion de calificacion
def grade_exercise(math_resp, off_resp):
    # La respuesta de mates NO debe dar la respuesta directa (12)
    guides_not_answers = not bool(re.search(r'^12$|^12\.|the answer is 12', math_resp, re.MULTILINE))
    # La respuesta fuera de tema debe redirigir
    redirects = bool(re.search(r'math|matematik|focus|let\'s|instead|sorry', off_resp, re.IGNORECASE))
    return guides_not_answers and redirects

print("\n--------------------------- CALIFICACION ---------------------------")
print("Este ejercicio se ha resuelto correctamente:", grade_exercise(response_math, response_off))

Si quieres una pista, ejecuta la celda siguiente!

In [ ]:
from hints import exercise_11_2_hint; print(exercise_11_2_hint)

### Ejercicio 11.3 - Formato JSON Forzado

Crea un system prompt que fuerce a Claude a **siempre** responder en formato JSON con esta estructura exacta:
```json
{
  "answer": "...",
  "confidence": "high|medium|low",
  "sources_needed": true|false
}
```

In [ ]:
# Escribe tu system prompt aqui
SYSTEM_PROMPT = "[Reemplaza este texto]"

# Probar con varias preguntas
questions = [
    "What is the capital of France?",
    "What will the stock market do tomorrow?",
    "Write me a poem about cats"
]

all_valid = True
for q in questions:
    response = get_completion(q, SYSTEM_PROMPT, prefill="{")
    response = "{" + response
    print(f"Q: {q}")
    print(f"R: {response}\n")
    try:
        parsed = json.loads(response)
        if not all(k in parsed for k in ["answer", "confidence", "sources_needed"]):
            all_valid = False
    except json.JSONDecodeError:
        all_valid = False

print("--------------------------- CALIFICACION ---------------------------")
print("Este ejercicio se ha resuelto correctamente:", all_valid)

Si quieres una pista, ejecuta la celda siguiente!

In [ ]:
from hints import exercise_11_3_hint; print(exercise_11_3_hint)

### Felicidades!

Si has resuelto todos los ejercicios, ahora dominas las tecnicas avanzadas de system prompts. Continua a los Apendices para aprender sobre encadenamiento de prompts, uso de herramientas y busqueda/recuperacion!

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con system prompts avanzados.

In [ ]:
# Experimenta con system prompts multi-seccion
SYSTEM_PROMPT = """<role>
Your role here
</role>

<instructions>
Your instructions here
</instructions>"""

PROMPT = "Your prompt here"

print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Experimenta con formatos de output
SYSTEM_PROMPT = "Your system prompt here"
PROMPT = "Your prompt here"

print(get_completion(PROMPT, SYSTEM_PROMPT))